In [ ]:
#!pip install tensorflow

In [ ]:
#! pip install transformers
#!pip install torch torchvision


In [ ]:
#!pip install tf-keras

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import nltk,re
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
true_df = pd.read_csv("/content/drive/MyDrive/News-_dataset/True.csv")
fake_df=pd.read_csv("/content/drive/MyDrive/News-_dataset/Fake.csv")

In [4]:
#Add label Column
true_df['label'] = 0
fake_df['label'] = 1
#true_df

In [5]:
true_df['Text'] = true_df['title'] + " " + true_df['text']

In [6]:
fake_df['Text'] = fake_df['title'] + " " + fake_df['text']

In [7]:
true_df=true_df.drop(['title','text','subject','date'],axis=1)
fake_df=fake_df.drop(['title','text','subject','date'],axis=1)

In [8]:
true_df_subset = true_df.loc[:1000]
fake_df_subset = fake_df.loc[:1000]
combined_df = pd.concat([true_df_subset, fake_df_subset], ignore_index=True)

In [9]:
combined_df.to_csv('combineddf2.csv', index=False)

In [10]:
combined_df = pd.read_csv('combineddf2.csv', header=None)

In [11]:
combined_df = combined_df.iloc[1:]

In [12]:
combined_df.head()

,0,1
1,0,"As U.S. budget fight looms, Republicans flip t..."
2,0,U.S. military to accept transgender recruits o...
3,0,Senior U.S. Republican senator: 'Let Mr. Muell...
4,0,FBI Russia probe helped by Australian diplomat...
5,0,Trump wants Postal Service to charge 'much mor...


In [13]:
print(combined_df.shape)

(2002, 2)


In [14]:
#Remove Reuters word From True News
def Remove_Reuters(text):
    regex = re.search(r"(?<=\(Reuters\)\s\-\s).*",text)
    if regex:
        return regex.group(0)
    return text
combined_df[1] = combined_df[1].apply(Remove_Reuters)

In [15]:
stop = stopwords.words('english')
def cleanText(txt):
  #lowercaing
    txt = txt.lower()
    txt = ' '.join([word for word in txt.split() if word not in (stop)])
  #removing encoding error sym
    txt = re.sub('[^a-z$0-9]', ' ', txt.lower())
    return txt
combined_df[1] = combined_df[1].apply(cleanText)

In [16]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel,ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [17]:
tokenized = combined_df[1].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True)))
print(tokenized)

1       [101, 2132, 4603, 3951, 10233, 1057, 1055, 351...
2       [101, 16824, 2111, 3039, 2034, 2051, 28845, 10...
3       [101, 2569, 9517, 4812, 6971, 3607, 2343, 8398...
4       [101, 8398, 3049, 11747, 2577, 13008, 3527, 24...
5       [101, 2343, 6221, 8398, 2170, 1057, 1055, 1069...
                              ...                        
1998    [101, 17180, 6356, 3867, 3020, 2740, 2729, 128...
1999    [101, 13055, 3477, 1002, 2322, 2243, 2095, 542...
2000    [101, 7078, 13675, 23496, 13966, 2617, 8398, 2...
2001    [101, 7349, 6398, 2768, 4532, 15876, 3600, 163...
2002    [101, 14726, 2047, 8554, 7657, 3795, 5448, 622...
Name: 1, Length: 2002, dtype: object


In [18]:
max_len = 128

padded = np.array([i[:max_len] + [0]*(max_len-len(i)) for i in tokenized.values])  # Pad sequences to max_len
attention_mask = np.where(padded != 0, 1, 0) 
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

In [19]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)  # last hidden state holds the output of bert

In [20]:
features = last_hidden_states[0][:,0,:].numpy()

In [21]:
labels = combined_df[0]

In [22]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, random_state=42)

In [23]:
#train the logistics regression model on the training set
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [24]:
from sklearn.metrics import classification_report
predictions = lr_clf.predict(test_features)
report = classification_report(test_labels, predictions)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93       307
           1       0.91      0.95      0.93       294

    accuracy                           0.93       601
   macro avg       0.93      0.93      0.93       601
weighted avg       0.93      0.93      0.93       601

